# Homework for week03

> We'll keep working with the MSRP variable, and we'll transform it to a classification task.

> In this homework, we will use the Car price dataset


In [44]:
# uri = 'https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-02-car-price/data.csv'

In [45]:
# !wget $uri -O data-hmwk-3.csv

In [46]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [47]:
data = pd.read_csv('data-hmwk-3.csv')
print(f'{data.shape =}')

data.shape =(11914, 16)


In [48]:
data.columns

Index(['Make', 'Model', 'Year', 'Engine Fuel Type', 'Engine HP',
       'Engine Cylinders', 'Transmission Type', 'Driven_Wheels',
       'Number of Doors', 'Market Category', 'Vehicle Size', 'Vehicle Style',
       'highway MPG', 'city mpg', 'Popularity', 'MSRP'],
      dtype='object')

In [49]:
data.columns = data.columns.str.replace('MSRP', 'price')

In [50]:
features = ['Make', 'Model', 'Year', 'Engine HP',
       'Engine Cylinders', 'Transmission Type', 'Vehicle Style',
       'highway MPG', 'city mpg']
target = ['price']

## Data Preparation

In [51]:
data = data[features+target]

In [52]:
data.columns = data.columns.str.replace(' ', '_').str.lower()

In [53]:
data.isnull().sum()

make                  0
model                 0
year                  0
engine_hp            69
engine_cylinders     30
transmission_type     0
vehicle_style         0
highway_mpg           0
city_mpg              0
price                 0
dtype: int64

In [54]:
data = data.fillna(0)

In [55]:
categorical_columns = list(data.dtypes[data.dtypes == 'object'].index)
numerical_columns = list(data.dtypes[data.dtypes != 'object'].index)

In [56]:
data.head().T

,0,1,2,3,4
make,BMW,BMW,BMW,BMW,BMW
model,1 Series M,1 Series,1 Series,1 Series,1 Series
year,2011,2011,2011,2011,2011
engine_hp,335.0,300.0,300.0,230.0,230.0
engine_cylinders,6.0,6.0,6.0,6.0,6.0
transmission_type,MANUAL,MANUAL,MANUAL,MANUAL,MANUAL
vehicle_style,Coupe,Convertible,Coupe,Coupe,Convertible
highway_mpg,26,28,28,28,28
city_mpg,19,19,20,18,18
price,46135,40650,36350,29450,34500


In [57]:
data.describe()

,year,engine_hp,engine_cylinders,highway_mpg,city_mpg,price
count,11914.000000,11914.000000,11914.000000,11914.000000,11914.000000,1.191400e+04
mean,2010.384338,247.941749,5.614655,26.637485,19.733255,4.059474e+04
std,7.579740,110.507669,1.800554,8.863001,8.987798,6.010910e+04
min,1990.000000,0.000000,0.000000,12.000000,7.000000,2.000000e+03
25%,2007.000000,170.000000,4.000000,22.000000,16.000000,2.100000e+04
50%,2015.000000,225.000000,6.000000,26.000000,18.000000,2.999500e+04
75%,2016.000000,300.000000,6.000000,30.000000,22.000000,4.223125e+04
max,2017.000000,1001.000000,16.000000,354.000000,137.000000,2.065902e+06


In [58]:
data

,make,model,year,engine_hp,engine_cylinders,transmission_type,vehicle_style,highway_mpg,city_mpg,price
0,BMW,1 Series M,2011,335.0,6.0,MANUAL,Coupe,26,19,46135
1,BMW,1 Series,2011,300.0,6.0,MANUAL,Convertible,28,19,40650
2,BMW,1 Series,2011,300.0,6.0,MANUAL,Coupe,28,20,36350
3,BMW,1 Series,2011,230.0,6.0,MANUAL,Coupe,28,18,29450
4,BMW,1 Series,2011,230.0,6.0,MANUAL,Convertible,28,18,34500
...,...,...,...,...,...,...,...,...,...,...
11909,Acura,ZDX,2012,300.0,6.0,AUTOMATIC,4dr Hatchback,23,16,46120
11910,Acura,ZDX,2012,300.0,6.0,AUTOMATIC,4dr Hatchback,23,16,56670
11911,Acura,ZDX,2012,300.0,6.0,AUTOMATIC,4dr Hatchback,23,16,50620
11912,Acura,ZDX,2013,300.0,6.0,AUTOMATIC,4dr Hatchback,23,16,50920


## Question 1

q: What is the most frequent observation (mode) for the column `transmission_type`?

a: `AUTOMATIC`

In [59]:
data.transmission_type.mode()

0    AUTOMATIC
Name: transmission_type, dtype: object

## Question 2

Create the correlation matrix for the numerical features of your dataset. In a correlation matrix, you compute the correlation coefficient between every pair of features in the dataset.

Q: What are the two features that have the biggest correlation in this dataset?

A: `highway_mpg` and `city_mpg`

In [60]:
corr = data[numerical_columns].corr()

In [61]:
corr

,year,engine_hp,engine_cylinders,highway_mpg,city_mpg,price
year,1.000000,0.338714,-0.040708,0.258240,0.198171,0.227590
engine_hp,0.338714,1.000000,0.774851,-0.415707,-0.424918,0.650095
engine_cylinders,-0.040708,0.774851,1.000000,-0.614541,-0.587306,0.526274
highway_mpg,0.258240,-0.415707,-0.614541,1.000000,0.886829,-0.160043
city_mpg,0.198171,-0.424918,-0.587306,0.886829,1.000000,-0.157676
price,0.227590,0.650095,0.526274,-0.160043,-0.157676,1.000000


In [62]:
import plotly.express as px

fig = px.imshow(corr, text_auto=".2f", aspect="auto", color_continuous_scale='RdBu_r')
fig.show()

### Make price binary

In [63]:
# calculate the mean for price column
mean_price = data.price.mean()
# create a new column called 'above_average' that is set to 1 if value is above the mean_price
data['above_average'] = [1 if price > mean_price else 0 for price in data['price']]

In [64]:
data.above_average.mean()

0.27438307873090484

### Split the data


In [65]:
from sklearn.model_selection import train_test_split

In [66]:
df_full_train, df_test = train_test_split(data, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)

In [67]:
len(df_train), len(df_val), len(df_test)

(7148, 2383, 2383)

In [68]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [69]:
y_train = df_train.above_average.values
y_val = df_val.above_average.values
y_test = df_test.above_average.values

df_train.drop(['price', 'above_average'], axis=1, inplace=True)
df_val.drop(['price', 'above_average'], axis=1, inplace=True)
df_test.drop(['price', 'above_average'], axis=1, inplace=True)

In [70]:
df_train.head()

,make,model,year,engine_hp,engine_cylinders,transmission_type,vehicle_style,highway_mpg,city_mpg
0,Mitsubishi,Endeavor,2011,225.0,6.0,AUTOMATIC,4dr SUV,19,15
1,Kia,Borrego,2009,276.0,6.0,AUTOMATIC,4dr SUV,21,17
2,Lamborghini,Gallardo,2012,570.0,10.0,MANUAL,Convertible,20,12
3,Chevrolet,Colorado,2016,200.0,4.0,AUTOMATIC,Crew Cab Pickup,27,20
4,Pontiac,Vibe,2009,158.0,4.0,AUTOMATIC,4dr Hatchback,26,20


In [71]:
y_train

array([0, 0, 1, ..., 0, 0, 0], dtype=int64)

## Question 3

- Calculate the mutual information score between above_average and other categorical variables in our dataset. Use the training set only.
- Round the scores to 2 decimals using round(score, 2).


Q: Which of these variables has the lowest mutual information score?

A: `transmission_type`

In [72]:
from sklearn.metrics import mutual_info_score

In [73]:
df_train[categorical_columns]

,make,model,transmission_type,vehicle_style
0,Mitsubishi,Endeavor,AUTOMATIC,4dr SUV
1,Kia,Borrego,AUTOMATIC,4dr SUV
2,Lamborghini,Gallardo,MANUAL,Convertible
3,Chevrolet,Colorado,AUTOMATIC,Crew Cab Pickup
4,Pontiac,Vibe,AUTOMATIC,4dr Hatchback
...,...,...,...,...
7143,Toyota,Sienna,AUTOMATIC,Passenger Minivan
7144,Chevrolet,HHR,MANUAL,Wagon
7145,Hyundai,Veracruz,AUTOMATIC,4dr SUV
7146,Mitsubishi,Expo,MANUAL,2dr Hatchback


In [74]:
def mutual_info_target(series):
    return mutual_info_score(series, df_full_train.above_average)

In [75]:
mi = df_full_train[categorical_columns].apply(mutual_info_target)
mi.sort_values(ascending=False)

model                0.460994
make                 0.238724
vehicle_style        0.083390
transmission_type    0.020884
dtype: float64

## Question 4

Q: What accuracy did you get?

A: `0.94`

In [76]:
from sklearn.feature_extraction import DictVectorizer

In [77]:
numerical_columns.remove('price')

In [78]:
dv = DictVectorizer(sparse=False)

train_dict = df_train.to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val.to_dict(orient='records')
X_val = dv.transform(val_dict)

In [79]:
from sklearn.linear_model import LogisticRegression

In [80]:
model = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

LogisticRegression(C=10, max_iter=1000, random_state=42, solver='liblinear')

In [81]:
y_pred = model.predict(X_val)
y_pred_proba = model.predict_proba(X_val)[:, 1]

In [82]:
pred_decision = (y_pred_proba >= 0.5)
(y_val == pred_decision).mean()

0.9441879983214435

In [83]:
from sklearn.metrics import accuracy_score
round(accuracy_score(y_val, y_pred), 2)

0.94

## Question 5

Q: Which of following feature has the smallest difference?

A: `city_mpg`

> [!Note] 
> :memo: I got the feature with smallest difference as `vehicle_style` with 0.04% difference as opposed to `city_mpg` with 0.21% (next smallest difference), but selected the smallest out of the MCQ options provided. 
> 
> :warning: Is my logic wrong, since my resulting feature is not in the options listed?

#### From the top

In [84]:
from sklearn.metrics import mean_squared_error


# load the data
data = pd.read_csv('data-hmwk-3.csv')

# identify features and target
features = ['make', 'model', 'year', 'engine_hp', 'engine_cylinders', 
            'transmission_type', 'vehicle_style',
            'highway_mpg', 'city_mpg']
target = ['price']

# clean/prepare the data
data.columns = data.columns.str.replace(' ', '_').str.lower()
data.columns = data.columns.str.replace('msrp', 'price')

# use a subset for analysis
data = data[features+target]

# impute Nan/nulls with 0
data = data.fillna(0)

categoricals = list(data.dtypes[data.dtypes == 'object'].index)
numericals = list(data.dtypes[data.dtypes != 'object'].index)


# binarize target
# calculate the mean for price column
mean_price = data.price.mean()
# create a new column called 'above_average' that is set to 1 if value is above the mean_price
data['above_average'] = [1 if price > mean_price else 0 for price in data['price']]


# split the data to train/val/test sets with 60%/20%/20% distribution
df_full_train, df_test = train_test_split(data, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)

# set the y dataframe
y_train = df_train.above_average.values
y_val = df_val.above_average.values

df_train.drop(['price', 'above_average'], axis=1, inplace=True)
df_val.drop(['price', 'above_average'], axis=1, inplace=True)


# perform OHE on categorical data
dv = DictVectorizer(sparse=False)

train_dict = df_train.to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val.to_dict(orient='records')
X_val = dv.transform(val_dict)


# Train a model with all the features
model = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

# Calculate the accuracy of the model
y_pred = model.predict(X_val)
# mse = mean_squared_error(y_val, y_pred)
# accuracy = 1 - (mse / np.var(y_val))
accuracy = round(model.score(X_val, y_val), 3)
print(f'The accuracy of the model with all features is {accuracy:.2%}')
# print(f'The accuracy of the model with all features is {model.score(X_val, y_val):.2%}')

The accuracy of the model with all features is 94.40%


In [85]:
# Exclude each feature from this set and train a model without it
for feature in df_train.columns:
    print(f'Removing {feature}')
    train_new = df_train.drop(feature, axis=1)
    val_new = df_val.drop(feature, axis=1)

    # display(train_new.head())

    # perform OHE on categorical data
    dv = DictVectorizer(sparse=False)

    train_dict_new = train_new.to_dict(orient='records')
    X_train_new = dv.fit_transform(train_dict_new)

    val_dict = val_new.to_dict(orient='records')
    X_val_new = dv.transform(val_dict)

    model_new = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)
    model_new.fit(X_train_new, y_train)
    y_pred_new = model_new.predict(X_val_new)
    # mse_new = mean_squared_error(y_val, y_pred_new)
    # new_accuracy = 1 - (mse_new / np.var(y_val))
    new_accuracy = round(model_new.score(X_val_new, y_val), 3)
    diff = round((accuracy - new_accuracy), 3)
    print(f'Number of features seen during fit = {model_new.n_features_in_}')
    print(f'old {accuracy = }')
    print(f'{new_accuracy = }')
    print(f'The difference in accuracy after excluding {feature} is {diff:.2%}')
    print()


Removing make
Number of features seen during fit = 895
old accuracy = 0.944
new_accuracy = 0.932
The difference in accuracy after excluding make is 1.20%

Removing model
Number of features seen during fit = 74
old accuracy = 0.944
new_accuracy = 0.924
The difference in accuracy after excluding model is 2.00%

Removing year
Number of features seen during fit = 942
old accuracy = 0.944
new_accuracy = 0.948
The difference in accuracy after excluding year is -0.40%

Removing engine_hp
Number of features seen during fit = 942
old accuracy = 0.944
new_accuracy = 0.923
The difference in accuracy after excluding engine_hp is 2.10%

Removing engine_cylinders


Number of features seen during fit = 942
old accuracy = 0.944
new_accuracy = 0.947
The difference in accuracy after excluding engine_cylinders is -0.30%

Removing transmission_type
Number of features seen during fit = 938
old accuracy = 0.944
new_accuracy = 0.939
The difference in accuracy after excluding transmission_type is 0.50%

Removing vehicle_style
Number of features seen during fit = 927
old accuracy = 0.944
new_accuracy = 0.944
The difference in accuracy after excluding vehicle_style is 0.00%

Removing highway_mpg
Number of features seen during fit = 942
old accuracy = 0.944
new_accuracy = 0.947
The difference in accuracy after excluding highway_mpg is -0.30%

Removing city_mpg
Number of features seen during fit = 942
old accuracy = 0.944
new_accuracy = 0.946
The difference in accuracy after excluding city_mpg is -0.20%



## Question 6

Q: Which of these alphas leads to the best RMSE on the validation set?

A: `0`


### From the top part 2, prepare_X_y()


In [86]:
def prepare_X_y():

    # load the data
    data = pd.read_csv('data-hmwk-3.csv')

    # identify features and target
    features = ['make', 'model', 'year', 'engine_hp', 'engine_cylinders', 
                'transmission_type', 'vehicle_style',
                'highway_mpg', 'city_mpg']
    target = ['price']

    # clean/prepare the data
    data.columns = data.columns.str.replace(' ', '_').str.lower()
    data.columns = data.columns.str.replace('msrp', 'price')

    # use a subset for analysis
    data = data[features+target]

    # impute Nan/nulls with 0
    data = data.fillna(0)

    # split the data to train/val/test sets with 60%/20%/20% distribution
    df_full_train, df_test = train_test_split(data, test_size=0.2, random_state=42)
    df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)

    # set the y dataframe
    y_train = np.log1p(df_train.price.values)
    y_val = np.log1p(df_val.price.values)

    # remove target from df
    del df_train['price']
    del df_val['price']


    # perform OHE on categorical data; made sparse due to non-convergence otherwise
    dv = DictVectorizer(sparse=True)

    train_dict = df_train.to_dict(orient='records')
    X_train = dv.fit_transform(train_dict)

    val_dict = df_val.to_dict(orient='records')
    X_val = dv.transform(val_dict)

    # print(X.shape, y.shape)
    return X_train, y_train, X_val, y_val


### Tuning `alpha`

In [87]:
from sklearn.linear_model import Ridge

rmse_scores = {}
alpha = [0, 0.01, 0.1, 1, 10]

X_train, y_train, X_val, y_val = prepare_X_y()

for num in alpha:
    model = Ridge(alpha=num, solver='sag', random_state=42)

    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)

    rmse = mean_squared_error(y_val, y_pred, squared=False)
    print(f'for alpha = {num}, rmse = {rmse:.3f}')
    rmse_scores[num] = rmse

print()
print(sorted(rmse_scores.items(), key=lambda x:x[1]))

for alpha = 0, rmse = 0.255
for alpha = 0.01, rmse = 0.255
for alpha = 0.1, rmse = 0.255
for alpha = 1, rmse = 0.258
for alpha = 10, rmse = 0.336

[(0, 0.2549331471185944), (0.01, 0.25494145671546464), (0.1, 0.255030890806678), (1, 0.2583191759168338), (10, 0.3360143230638771)]
